In [9]:
import pandas as pd

In [10]:
df= pd.read_csv('shared/complaints_25Nov21.csv')

In [4]:
df

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,2016-10-26,Money transfers,International money transfer,Other transaction issues,NaN,"To whom it concerns, I would like to file a fo...",Company has responded to the consumer and the ...,"CITIBANK, N.A.",NaN,NaN,NaN,Consent provided,Web,2016-10-29,Closed with explanation,Yes,No,2180490
1,2015-03-27,Bank account or service,Other bank product/service,"Account opening, closing, or management",NaN,My name is XXXX XXXX XXXX and huband name is X...,Company chooses not to provide a public response,"CITIBANK, N.A.",PA,151XX,Older American,Consent provided,Web,2015-03-27,Closed with explanation,Yes,No,1305453
2,2015-04-20,Bank account or service,Other bank product/service,"Making/receiving payments, sending money",NaN,XXXX 2015 : I called to make a payment on XXXX...,Company chooses not to provide a public response,U.S. BANCORP,PA,152XX,NaN,Consent provided,Web,2015-04-22,Closed with monetary relief,Yes,No,1337613
3,2013-04-29,Mortgage,Conventional fixed mortgage,"Application, originator, mortgage broker",NaN,NaN,NaN,JPMORGAN CHASE & CO.,VA,22406,Servicemember,NaN,Phone,2013-04-30,Closed with explanation,Yes,Yes,393900
4,2013-05-29,Mortgage,Other mortgage,"Loan modification,collection,foreclosure",NaN,NaN,NaN,"BANK OF AMERICA, NATIONAL ASSOCIATION",GA,30044,NaN,NaN,Referral,2013-05-31,Closed with explanation,Yes,No,418647
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207255,2015-05-24,Debt collection,Credit card,Taking/threatening an illegal action,Sued w/o proper notification of suit,NaN,NaN,JPMORGAN CHASE & CO.,FL,33133,NaN,Consent not provided,Web,2015-05-24,Closed with explanation,Yes,No,1390395
207256,2012-01-10,Mortgage,Conventional fixed mortgage,"Loan modification,collection,foreclosure",NaN,NaN,NaN,JPMORGAN CHASE & CO.,NY,10312,NaN,NaN,Referral,2012-01-11,Closed without relief,Yes,Yes,12192
207257,2012-07-17,Student loan,Non-federal student loan,Repaying your loan,NaN,NaN,NaN,"BANK OF AMERICA, NATIONAL ASSOCIATION",NH,032XX,NaN,NaN,Web,2012-07-18,Closed with explanation,Yes,No,118351
207258,2016-09-29,Bank account or service,Checking account,"Account opening, closing, or management",NaN,Near the end of XXXX 2016 I opened a Citigold ...,Company has responded to the consumer and the ...,"CITIBANK, N.A.",CA,900XX,NaN,Consent provided,Web,2016-09-29,Closed with non-monetary relief,Yes,No,2138969


In [11]:

# Define your predictors (X) using the specified columns
selected_columns = ['Product', 'Sub-product', 'Issue', 'State', 'Tags', 'Submitted via', 'Company response to consumer', 'Timely response?']
X = df[selected_columns]

from sklearn import preprocessing

# Convert 'Consumer disputed?' to 0s and 1s using label encoding
le = preprocessing.LabelEncoder()
y = le.fit_transform(df['Consumer disputed?'])


In [12]:
from sklearn.model_selection import train_test_split

# Split the data into a training and testing set (80/20 split)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)


In [13]:
from imblearn.under_sampling import RandomUnderSampler

# Calculate the proportion of complaints that are disputed in the training dataset
proportion_disputed = sum(y_train) / len(y_train)
print (proportion_disputed)

# Check if the proportion is less than 30%
if proportion_disputed < 0.3:
    # Apply random undersampling to balance the dataset
    undersampler = RandomUnderSampler(random_state=123)
    X_train, y_train = undersampler.fit_resample(X_train, y_train)


0.21684719675769565


In [14]:
from sklearn.preprocessing import OneHotEncoder
# Import XGBoost's XGBClassifier
from xgboost import XGBClassifier

# Create an instance of XGBClassifier
xgb = XGBClassifier()

# Now you can use xgb for your machine learning tasks


# Initialize the XGBoost Classifier with random state
model_xgb = XGBClassifier(random_state=123)

# Define the categorical columns to be one-hot encoded
categorical_cols = ['Product', 'Sub-product', 'Issue', 'State', 'Tags', 'Submitted via', 'Company response to consumer', 'Timely response?']

# Create a one-hot encoder that handles unknown categories
one_hot_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)

# Fit the encoder on the training data and transform both training and test data
X_train_encoded = one_hot_encoder.fit_transform(X_train[categorical_cols])
X_test_encoded = one_hot_encoder.transform(X_test[categorical_cols])

# Concatenate the one-hot encoded features with the non-categorical features
import numpy as np
X_train_encoded = np.concatenate((X_train_encoded, X_train.drop(categorical_cols, axis=1)), axis=1)
X_test_encoded = np.concatenate((X_test_encoded, X_test.drop(categorical_cols, axis=1)), axis=1)

# Fit the model on the training data
model_xgb.fit(X_train_encoded, y_train)

# Make predictions on the test set
y_pred = model_xgb.predict(X_test_encoded)


/opt/conda/envs/mggy8413/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [19]:
from sklearn.metrics import classification_report, confusion_matrix

# Create a classification report
class_report = classification_report(y_test, y_pred)

# Create a confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)

print("Classification Report:\n", class_report)
print("\nConfusion Matrix:\n", conf_matrix)


Classification Report:
               precision    recall  f1-score   support

           0       0.84      0.53      0.65     32504
           1       0.27      0.63      0.38      8948

    accuracy                           0.55     41452
   macro avg       0.55      0.58      0.51     41452
weighted avg       0.72      0.55      0.59     41452


Confusion Matrix:
 [[17128 15376]
 [ 3302  5646]]


In [20]:
# Define the cost structure
cost_disputed = 600
cost_non_disputed = 100
extra_due_diligence_cost = 90

# Calculate the total cost for the test set
total_cost = (y_test == 1).sum() * cost_disputed + (y_test == 0).sum() * cost_non_disputed + (y_test == 1).sum() * extra_due_diligence_cost

# Print the total cost
print(f"Total cost for the test set: ${total_cost}")


Total cost for the test set: $9424520


In [21]:
# Define the cost structure
cost_disputed = 600
cost_non_disputed = 100
extra_due_diligence_cost = 90

# Extract the values from the confusion matrix
tn, fp, fn, tp = conf_matrix.ravel()

# Calculate the total cost based on the confusion matrix
total_cost_model = tp * cost_disputed + tn * cost_non_disputed + fn * cost_disputed + extra_due_diligence_cost * (fp + fn)

# Print the total cost based on the model results
print(f"Total cost based on the model results: ${total_cost_model}")


Total cost based on the model results: $8762620


In [22]:
#Question1

# Calculate the proportion of consumers who raised a dispute in the test set
proportion_disputed_test = (y_test == 1).sum() / len(y_test)

# Print the proportion
print(f"Proportion of consumers who raised a dispute in the test set: {proportion_disputed_test:.2%}")


Proportion of consumers who raised a dispute in the test set: 21.59%


In [23]:
#Question2

# Calculate the proportion of consumers who raised a dispute in the modified training set
proportion_disputed_train = (y_train == 1).sum() / len(y_train)

# Print the proportion
print(f"Proportion of consumers who raised a dispute in the modified training set: {proportion_disputed_train:.2%}")


Proportion of consumers who raised a dispute in the modified training set: 50.00%


In [24]:
#3
from xgboost import XGBClassifier
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import recall_score

# Define the cost structure
cost_disputed = 600
cost_non_disputed = 100
extra_due_diligence_cost = 90

# Define the categorical columns to be one-hot encoded
categorical_cols = ['Product', 'Sub-product', 'Issue', 'State', 'Tags', 'Submitted via', 'Company response to consumer', 'Timely response?']

# Define the remaining non-categorical columns
non_categorical_cols = [col for col in X_train.columns if col not in categorical_cols]

# Create a column transformer to apply one-hot encoding to the categorical columns
preprocessor = ColumnTransformer(
    transformers=[('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)],
    remainder='passthrough'  # Pass through the non-categorical columns
)

# Create a pipeline to combine preprocessor and the XGBoost classifier
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', XGBClassifier(random_state=123))
])

# Fit the model on the training data
pipeline.fit(X_train, y_train)

# Make predictions on the test set
y_pred = pipeline.predict(X_test)

# Calculate the recall for the category 'Consumer disputed?' = 'Yes'
recall_consumer_disputed_yes = recall_score(y_test, y_pred, pos_label=1)

print(f"Recall for 'Consumer disputed?' = 'Yes' on the test set: {recall_consumer_disputed_yes:.2f}")


Recall for 'Consumer disputed?' = 'Yes' on the test set: 0.63


In [26]:
#4
# Define the cost structure
cost_disputed = 600
cost_non_disputed = 100
extra_due_diligence_cost = 90

# Calculate the total cost for the test set without a model
total_cost_no_model = (y_test == 1).sum() * cost_disputed + (y_test == 0).sum() * cost_non_disputed 
print(f"Total cost for the test set without a model: ${total_cost_no_model}")


Total cost for the test set without a model: $8619200


In [27]:
#5
# Calculate the cost for each complaint based on the model's predictions
cost_per_complaint = np.where(y_pred == 1, extra_due_diligence_cost, cost_non_disputed)

# Calculate the total cost based on the cost per complaint
total_cost_with_model = sum(cost_per_complaint)

print(f"Total cost to the banks with the model: ${total_cost_with_model}")



Total cost to the banks with the model: $3934980


In [35]:
# Calculate the confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Calculate total cost
true_negatives, false_positives, false_negatives, true_positives = cm.ravel()

cost_with_model = (true_negatives*100) + ((true_positives+false_positives)*(100+90)) + (false_negatives*600)

print(f"Estimated total cost with a model : ${cost_with_model}")
print(cm)


Estimated total cost with a model : $7688180
[[17128 15376]
 [ 3302  5646]]


In [29]:
#6
import numpy as np

# Define the cost structure
cost_disputed = 600
cost_non_disputed = 100
extra_due_diligence_cost = 90

# Define the categorical columns to be one-hot encoded
categorical_cols = ['Product', 'Sub-product', 'Issue', 'State', 'Tags', 'Submitted via', 'Company response to consumer', 'Timely response?']

# Define the remaining non-categorical columns
non_categorical_cols = [col for col in X_train.columns if col not in categorical_cols]

# Create a column transformer to apply one-hot encoding to the categorical columns
preprocessor = ColumnTransformer(
    transformers=[('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)],
    remainder='passthrough'  # Pass through the non-categorical columns
)

# Create a pipeline to combine preprocessor and the XGBoost classifier
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', XGBClassifier(random_state=123))
])

# Fit the model on the training data
pipeline.fit(X_train, y_train)

# Define a range of thresholds to test
thresholds = np.arange(0.1, 1.0, 0.05)

# Initialize variables to keep track of the best threshold and the lowest total cost
best_threshold = 0
lowest_cost = float('inf')

# Calculate the total cost for each threshold
for threshold in thresholds:
    # Apply the current threshold to the model's predicted probabilities
    y_pred_threshold = (pipeline.predict_proba(X_test)[:, 1] > threshold).astype(int)
    
    # Calculate the cost for each complaint based on the threshold
    cost_per_complaint = np.where(y_pred_threshold == 1, extra_due_diligence_cost, cost_disputed)
    
    # Calculate the total cost based on the cost per complaint
    total_cost_threshold = sum(cost_per_complaint)
    
    # Check if the total cost with this threshold is the lowest so far
    if total_cost_threshold < lowest_cost:
        lowest_cost = total_cost_threshold
        best_threshold = threshold

print(f"Optimal threshold for lowest total cost: {best_threshold:.2f}")
print(f"Lowest total cost achieved: ${lowest_cost}")


Optimal threshold for lowest total cost: 0.10
Lowest total cost achieved: $3792900


In [30]:
# Create a classification report
class_report = classification_report(y_test, y_pred)

# Create a confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)

print("Classification Report:\n", class_report)
print("\nConfusion Matrix:\n", conf_matrix)


Classification Report:
               precision    recall  f1-score   support

           0       0.84      0.53      0.65     32504
           1       0.27      0.63      0.38      8948

    accuracy                           0.55     41452
   macro avg       0.55      0.58      0.51     41452
weighted avg       0.72      0.55      0.59     41452


Confusion Matrix:
 [[17128 15376]
 [ 3302  5646]]
